In [ ]:
# import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains, Chrome
import time

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
#!pip install selenium

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df_all=pd.DataFrame()
#code=['E445167','E445186','E446807']


In [ ]:
item_list=pd.read_excel('/content/gdrive/My Drive/202203_Project/item_list.xlsx')

In [ ]:
item_list.shape

(100, 11)

In [ ]:

!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graph

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
URL_List=[]
for item in item_list['item code']:
    #get subpage data
    if item_list[item_list['item code']==item]['url'].values[0] not in URL_List:
        URL = item_list[item_list['item code']==item]['url'].values[0]
    else:
        URL = item_list[item_list['item code']==item]['url'].values[1]
    URL_List.append(URL)
    print(URL)
    driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

    driver.get(URL) #replacing the requests.get step

    #get the code list of available unit trusts
    soup = BeautifulSoup(driver.page_source, "html.parser")
    soup.prettify()
    
    #Click all もっと見る
    click=0
    click=0
    try:
        python_button = driver.find_elements_by_xpath("//*[@id='root']/div/section/div[3]/section/div/div[2]/div[2]/button")[0]
        while ('もっと見る' == soup.find_all(class_="uq-ec-show-more uq-ec-button-reset uq-ec-button--large uq-ec-cursor-pointer uq-ec-mb-spacing-06")[0].text)and(click<=200):
            click+=1
            python_button.click()
            soup = BeautifulSoup(driver.page_source, "html.parser")
            soup.prettify()
    except:
        pass
    
    
    dict_df={}
    count=0
    for i in soup.find_all(class_="uq-ec-caption uq-ec-caption--standard-color-primary-dark uq-ec-text-align-left uq-ec-review__user-info uq-ec-review__grey-text uq-ec-caption--standard uq-ec-text-transform-normal"):
        if ('性' in i.text)or ('回答しない'in i.text) or ('その他'in i.text):
            if count != 0:    
                while len(dict_df['{}_{}'.format(item,count)]) < 5:
                    dict_df['{}_{}'.format(item,count)].append('nan')
                    
            count+=1
            dict_df['{}_{}'.format(item,count)]=[]
            dict_df['{}_{}'.format(item,count)].append(i.text)
        elif ('代' in i.text)|('歳' in i.text)|('ヶ月'in i.text):
            dict_df['{}_{}'.format(item,count)].append(i.text)
        elif '身長' in i.text:
            while len(dict_df['{}_{}'.format(item,count)]) <2:
                dict_df['{}_{}'.format(item,count)].append('nan')
            dict_df['{}_{}'.format(item,count)].append(i.text)
        elif '体重' in i.text:
            while len(dict_df['{}_{}'.format(item,count)]) <3:
                dict_df['{}_{}'.format(item,count)].append('nan')
            dict_df['{}_{}'.format(item,count)].append(i.text)
        elif '足のサイズ' in i.text:
            while len(dict_df['{}_{}'.format(item,count)]) <4:
                dict_df['{}_{}'.format(item,count)].append('nan')
            dict_df['{}_{}'.format(item,count)].append(i.text)


    while len(dict_df['{}_{}'.format(item,count)]) < 5:
        dict_df['{}_{}'.format(item,count)].append('nan')
    
    df=pd.DataFrame(dict_df)
    
    # Size and Rating
    dict2={'Size':[],'Rating':[],'Item_Code':[]}
    for i in soup.find_all(class_="uq-ec-caption uq-ec-caption--standard-color-primary-dark uq-ec-text-align-left uq-ec-review__purchase--information uq-ec-caption--standard uq-ec-text-transform-normal"):
      dict2['Size'].append(i.text)
      dict2['Item_Code'].append(item)
    
    for i in soup.find_all(class_="uq-ec-rating-static uq-ec-rating"):
      dict2['Rating'].append(str(i).count("full"))
    
    
    df2=pd.DataFrame(dict2)
    
    df1=df.transpose()
    df1.reset_index(inplace=True)
    
    df_new=pd.concat([df1,df2], axis=1)
    
    df_all=pd.concat([df_all,df_new],axis=0)
    driver.close()



In [ ]:
dict_df

In [ ]:
df_all.columns=['id','gender','AgeGroup','Height','Weight','Foot_Size','Product_Size','Rating','Item_Code']

In [ ]:
df_all

In [ ]:
length_dict = {key: len(value) for key, value in dict_df.items()}
length_dict

In [ ]:
df_all.to_csv('/content/gdrive/My Drive/202203_Project/all_reviews.csv', encoding='utf-8', index=False)